**--------------------------------- IMPORTING PACKAGES --------------------------------------**

**datetime - saves data in time and/or date format**

**seaborn - data visualization (ie heatmap)**

**time - convert time values to hour format (ie 10:50 (10 hr 50 min) converted to 10.833 hrs)**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import plotly.plotly as py
import plotly.graph_objs as go
import datetime as dt
import time
import matplotlib.gridspec as gridspec
#pd.options.display.float_format = '[:,.2f]'
from mpldatacursor import datacursor

**--------------------------------- OPENING DATA FILE --------------------------------------**


**Data obtained from https://perso.telecom-paristech.fr/eagan/class/igr204/datasets**

In [2]:
df = pd.read_csv('https://perso.telecom-paristech.fr/eagan/class/igr204/data/TimeUse.csv')
display(HTML(df.to_html()))

,SEX,GEO/ACL00,Total,Personal care,Sleep,Eating,Other and/or unspecified personal care,"Employment, related activities and travel as part of/during main and second job",Main and second job and related travel,Activities related to employment and unspecified employment,Study,School and university except homework,Homework,Free time study,Household and family care,Food management except dish washing,Dish washing,Cleaning dwelling,Household upkeep except cleaning dwelling,Laundry,Ironing,Handicraft and producing textiles and other care for textiles,Gardening; other pet care,Tending domestic animals,Caring for pets,Walking the dog,Construction and repairs,Shopping and services,"Childcare, except teaching, reading and talking","Teaching, reading and talking with child",Household management and help family member,"Leisure, social and associative life",Organisational work,Informal help to other households,Participatory activities,Visiting and feasts,Other social life,Entertainment and culture,Resting,Walking and hiking,Sports and outdoor activities except walking and hiking,Computer games,Computing,Hobbies and games except computing and computer games,Reading books,"Reading, except books",TV and video,Radio and music,Unspecified leisure,Travel except travel related to jobs,Travel to/from work,Travel related to study,Travel related to shopping and services,Transporting a child,Travel related to other household purposes,"Travel related to leisure, social and associative life",Unspecified travel,Unspecified time use
0,Males,Belgium,24:00,10:45,8:15,1:49,0:42,3:07,3:05,0:02,0:11,0:05,0:03,0:03,2:28,0:22,0:10,0:08,0:18,0:01,0:01,0:00,0:19,0:00,0:03,0:05,0:19,0:24,0:05,0:04,0:08,5:58,0:07,0:00,0:03,0:32,0:23,0:10,0:27,0:12,0:15,0:05,0:22,0:13,0:06,0:22,2:35,0:05,0:01,1:30,0:25,0:02,0:16,0:03,0:00,0:15,0:30,0:01
1,Males,Bulgaria,24:00,11:54,9:08,2:07,0:39,3:32,3:27,0:04,0:03,0:02,0:01,0:00,2:37,0:15,0:05,0:06,0:22,0:01,0:00,0:00,0:36,0:32,0:01,0:02,0:16,0:13,0:02,0:05,0:01,4:46,0:00,0:09,0:01,0:04,0:37,0:01,0:10,0:16,0:10,0:00,0:01,0:11,0:06,0:15,2:41,0:06,0:01,1:07,0:23,0:00,0:12,0:01,0:06,0:21,0:03,0:02
2,Males,Germany (including former GDR from 1991),24:00,10:40,8:08,1:43,0:49,3:27,3:21,0:06,0:15,0:06,0:05,0:04,2:22,0:16,0:08,0:11,0:14,0:02,0:01,0:00,0:17,0:01,0:03,0:03,0:19,0:29,0:05,0:05,0:09,5:42,0:09,0:08,0:04,0:17,0:45,0:14,0:16,0:13,0:15,0:05,0:16,0:18,0:06,0:31,1:58,0:05,0:00,1:29,0:27,0:02,0:16,0:02,0:05,0:34,0:03,0:05
3,Males,Estonia,24:00,10:35,8:24,1:19,0:52,4:27,4:20,0:07,0:06,0:03,0:02,0:02,2:33,0:21,0:06,0:09,0:22,0:01,0:00,0:01,0:16,0:04,0:01,0:05,0:29,0:20,0:06,0:04,0:06,5:02,0:02,0:15,0:01,0:04,0:26,0:05,0:21,0:10,0:13,0:01,0:02,0:05,0:14,0:23,2:29,0:11,0:00,1:12,0:28,0:01,0:13,0:01,0:07,0:22,0:01,0:04
4,Males,Spain,24:00,11:11,8:36,1:47,0:48,4:21,4:17,0:03,0:18,0:06,0:07,0:04,1:37,0:19,0:04,0:07,0:06,0:01,0:00,0:00,0:09,0:03,0:01,0:03,0:06,0:19,0:07,0:04,0:06,5:16,0:01,0:07,0:03,0:12,0:45,0:07,0:24,0:39,0:14,0:02,0:09,0:10,0:04,0:13,2:00,0:05,0:00,1:16,0:31,0:02,0:07,0:02,0:03,0:28,0:02,0:02
5,Males,France,24:00,11:44,8:45,2:18,0:41,3:48,3:46,0:02,0:15,0:09,0:05,0:01,2:24,0:16,0:08,0:11,0:08,0:01,0:01,0:00,0:18,0:03,0:05,:,0:32,0:30,0:05,0:04,0:04,4:44,0:01,0:10,0:07,0:21,0:20,0:05,0:06,0:20,0:17,:,0:07,0:14,0:01,0:22,2:08,0:04,0:00,1:03,0:24,0:02,:,0:02,:,:,0:35,0:02
6,Males,Italy,24:00,11:16,8:17,1:57,1:02,4:15,4:11,0:04,0:11,0:04,0:06,0:01,1:35,0:11,0:05,0:09,0:06,0:00,0:00,0:00,0:16,0:01,0:01,0:02,0:06,0:22,0:04,0:07,0:04,5:05,0:02,0:07,0:04,0:17,0:42,0:06,0:32,0:23,0:15,0:02,0:07,0:12,0:04,0:17,1:52,0:04,0:00,1:35,0:32,0:02,0:12,0:02,0:03,0:36,0:08,0:03
7,Males,Latvia,24:00,10:46,8:35,1:33,0:37,5:00,4:55,0:06,0:09,0:05,0:02,0:02,1:50,0:16,0:04,0:06,0:21,0:01,0:00,0:00,0:17,0:03,0:01,0:03,0:17,0:12,0:02,0:02,0:06,4:45,0:00,0:11,0:01,0:20,0:14,0:05,0:23,0:12,0:19,0:02,0:03,0:05,0:09,0:17,2:18,0:06,0:00,1:28,0:37,0:02,0:12,0:01,0:07,0:26,0:02,0:02
8,Males,Lithuania,24:00,10:53,8:28,1:32,0:53,4:45,4:43,0:03,0:09,0:07,0:0

**----------------------------- USING BASIC FUNCTIONS TO GATHER DATA FROM SET --------------------------------------**


In [3]:
df.describe()

,SEX,GEO/ACL00,Total,Personal care,Sleep,Eating,Other and/or unspecified personal care,"Employment, related activities and travel as part of/during main and second job",Main and second job and related travel,Activities related to employment and unspecified employment,...,Unspecified leisure,Travel except travel related to jobs,Travel to/from work,Travel related to study,Travel related to shopping and services,Transporting a child,Travel related to other household purposes,"Travel related to leisure, social and associative life",Unspecified travel,Unspecified time use
count,28,28,28,28,28,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28
unique,2,14,1,26,23,22,20,27,27,7,...,3,22,18,4,12,7,9,18,10,8
top,Males,Bulgaria,24:00,11:11,8:35,1:26,0:41,3:48,3:46,0:03,...,0:00,1:30,0:18,0:02,0:14,0:03,0:03,0:36,0:01,0:02
freq,14,2,28,2,3,4,3,2,2,7,...,23,2,3,16,5,8,8,3,9,9


In [ ]:
print (df.isnull().sum())

In [ ]:
df.info()

In [ ]:
print(df.dtypes)                                        

In [ ]:
df.columns

**--------------------------------- CLEANING DATA TO BE IN READABLE FORMAT --------------------------------------**


**(1) TIME: Converting data presented in table (HH:MM) to float value for analysis**

**Method of conversion: (HH:MM) --> add seconds column (HH:MM:SS) --> convert to decimal format in hours**

**(2) COLUMN NAMES: Combining column names with same name (ie multiple 'Travel')**

**Method: reducing column name to first word, can do count_values to find if column name is used more than once**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#(1) CONVERT DATA TIMES TO DECIMAL VALUES

for i in df.columns:
    if i=='SEX' or i=='GEO/ACL00' or i=='Total':                   #1st 3 columns leave as category type
        df[i]=df[i].str.strip().astype('category')                 #outputs not integars
    else:
        for j in range(len(df[i])):                                #sets empty cells to 00:00 (HH:MM)
            if df.iloc[j][i]==':':
                df.set_value(j,i,"00:00")                             
        df[i]=df[i].astype(str)+':00'                              #for all cells, convert values to str and append ':00' for (SS) 

    
        df[i]=pd.to_timedelta(df[i],'s')                           #read values as HH:MM:SS    
        df[i]=df[i].dt.seconds/3600.0                              #divide time by number of sec to get decimal in hours
        df[i]=df[i].round(2)                                       #round values to 2 sig figs after decimal
        #print(df[i])
            

display(HTML(df.to_html()))
df.dtypes                                                          #ensure that all columns except 1st 3 are float

In [ ]:
#(2) COLUMN NAMES

new_columns=[]
for i in range(len(df.columns)):                           #split column name to only 1st word; remove "," or ";" at end of words if present
    new_columns.append(df.columns[:][i].split(' ')[0].strip(',').strip(';'))

df.columns = new_columns
df['Travel_sum']=df.Travel.sum(axis=1)                     #combine "Travel", "Unspecified", and "Reading" since multiple found
df=df.drop(['Travel'], axis=1)
df['Unspecified_sum']=df.Unspecified.sum(axis=1)
df=df.drop(['Unspecified'], axis=1)
df['Reading_sum']=df.Reading.sum(axis=1)
df=df.drop(['Reading'], axis=1)
df.head()


In [ ]:
#GROUPING BY COUNTRY NAME

group1 = df.groupby(["GEO/ACL00"]).agg(np.mean)
display(HTML(group1.to_html()))

**--------------------------------- DATA VISUALIZATION --------------------------------------**

**Plotting and exploring pie charts**

In [ ]:
df.plot()
plt.title('Plot of All Data', fontweight='bold')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)    #place legend box to right of plot
plt.show()

In [ ]:
df.plot()
plt.yscale('log')                                                 # makes plot log scale 


plt.title('Log Plot of All Data', fontweight='bold')              # figure features
plt.xlabel('Index')
plt.ylabel('Log Scale')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.) 
plt.show()

In [ ]:
#SINGLE PIE CHART

plt.pie(
    df.iloc[1,3:],
    labels=df.columns[3:],
    shadow=False,
    startangle=90,
    # with the percent listed as a fraction
    autopct='%1.1f%%',
    )
# View the plot drop above
plt.axis('equal')

# View the plot
plt.tight_layout()
plt.title('Male Time Use in Belgium', fontweight='bold')
plt.show()

In [ ]:
#MULTIPLE PIE CHART SUBPLOTS FOR ALL COUNTRIES


#getting data
data1 = group1.iloc[0,3:]
data2 = group1.iloc[1,3:]
data3 = group1.iloc[2,3:]
data4 = group1.iloc[3,3:]
data5 = group1.iloc[4,3:]
data6 = group1.iloc[5,3:]
data7 = group1.iloc[6,3:]
data8 = group1.iloc[7,3:]
data9 = group1.iloc[8,3:]
data10 = group1.iloc[9,3:]
data11 = group1.iloc[10,3:]
data12 = group1.iloc[12,3:]
data13 = group1.iloc[12,3:]
data14 = group1.iloc[13,3:]


# create a figure with two subplots
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10), (ax11, ax12), (ax13, ax14)) = plt.subplots(7, 2, figsize = (70,70), subplot_kw={'aspect':'equal'})



# plot each pie chart in a separate subplot 
ax1.pie(data1)
ax2.pie(data2)
ax3.pie(data3)
ax4.pie(data4)
ax5.pie(data5)
ax6.pie(data6)
ax7.pie(data7)
ax8.pie(data8)
ax9.pie(data9)
ax10.pie(data10)
ax11.pie(data11)
ax12.pie(data12)
ax13.pie(data13)
ax14.pie(data14)


#subplot titles
ax1.set_title('Belgium', fontweight = 'bold', fontsize=30)
ax2.set_title('Bulgaria', fontweight = 'bold', fontsize=30)
ax3.set_title('Estonia', fontweight = 'bold', fontsize=30)
ax4.set_title('Finland', fontweight = 'bold', fontsize=30)
ax5.set_title('France', fontweight = 'bold', fontsize=30)
ax6.set_title('Germany', fontweight = 'bold', fontsize=30)
ax7.set_title('Italy', fontweight = 'bold', fontsize=30)
ax8.set_title('Latvia', fontweight = 'bold', fontsize=30)
ax9.set_title('Lithuania', fontweight = 'bold', fontsize=30)
ax10.set_title('Norway', fontweight = 'bold', fontsize=30)
ax11.set_title('Poland', fontweight = 'bold', fontsize=30)
ax12.set_title('Slovenia', fontweight = 'bold', fontsize=30)
ax13.set_title('Spain', fontweight = 'bold', fontsize=30)
ax14.set_title('United Kingdom', fontweight = 'bold', fontsize=30)


datacursor(hover=True, point_labels=df['Employment'])

#, point_labels=df['E']
plt.legend(bbox_to_anchor=(1.05, 1), loc='best', borderaxespad=0.5,prop={'size':20}, labels= group1.columns[3:]) 
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
plt.tight_layout()
plt.show()

In [ ]:
#MULTIPLE PIE CHART SUBPLOTS FOR ALL COUNTRIES -- ALTERNATIVES


"""
#WORKING CODE FOR PLOTTING ALL COUNTRY DATA; BULKY RESULT

for i in range(len(group1)):    
    plt.figure(figsize=(10,7))
    plt.pie(group1.iloc[i,3:],
        labels= group1.columns[3:],
        #autopct='%1.1f%%',
        rotatelabels=True)
    a='Time Use in '+ group1.index[i]
    plt.title(a, fontweight = 'bold')
       
    # View the plot        
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
    

"""




In [ ]:
#heatmap
f,ax = plt.subplots(figsize=(18, 16))
sns.heatmap(df.corr(), annot=True, linewidths=.8, fmt= '.1f',ax=ax)
plt.show()

**--------------------------------- ISSUES --------------------------------------**
- **column name including "/" because wasn't able to group_by_country; had difficulty renaming**
- **setting pie chart subplots with necessary data and adjusting frame size**